# Improved CIFAR 10 accuracy using CNN

In [1]:
%pip install keras-tuner

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.1/129.1 kB 6.9 MB/s eta 0:00:00


In [2]:
import tensorflow as tf
import keras_tuner as kt
import numpy as np
import matplotlib.pyplot as plt

from functools import partial

## Load CIFAR 10 dataset

In [12]:
cifar_10 = tf.keras.datasets.cifar10.load_data()
(train_images, train_labels), (test_images, test_labels) = cifar_10

# Scale pixel intensities (immediate 10x improvement to accuracy)

train_images = train_images / 255.0
test_images = test_images / 255.0

train_images.shape

(50000, 32, 32, 3)

## Create Model

In [13]:
def build_model():
    DefaultConv2D = partial(tf.keras.layers.Conv2D,
                           padding="same",
                           activation='relu',
                           kernel_initializer='he_normal')


    learning_rate = 0.01
    optimizer = tf.keras.optimizers.SGD(learning_rate=learning_rate)

    model = tf.keras.Sequential()

    # Initial convolutional layer
    model.add(DefaultConv2D(filters = 64, kernel_size = 3, input_shape=[32, 32, 3]))

    model.add(tf.keras.layers.BatchNormalization())


    # More convolutional layers
    model.add(DefaultConv2D(filters = 128, kernel_size = 3, input_shape=[32, 32, 3]))

    model.add(DefaultConv2D(filters = 128, kernel_size = 3, input_shape=[32, 32, 3]))

    model.add(tf.keras.layers.MaxPool2D(pool_size=2))

    model.add(DefaultConv2D(filters = 256, kernel_size = 3, input_shape=[32, 32, 3]))

    model.add(tf.keras.layers.BatchNormalization())

    model.add(DefaultConv2D(filters = 256, kernel_size = 3, input_shape=[32, 32, 3]))


    model.add(DefaultConv2D(filters = 256, kernel_size = 3, input_shape=[32, 32, 3]))

    model.add(tf.keras.layers.MaxPool2D(pool_size=2))
    model.add(tf.keras.layers.Dropout(0.2))

    model.add(DefaultConv2D(filters = 512, kernel_size = 3, input_shape=[32, 32, 3]))

    model.add(tf.keras.layers.BatchNormalization())

    model.add(DefaultConv2D(filters = 512, kernel_size = 3, input_shape=[32, 32, 3]))

    model.add(DefaultConv2D(filters = 512, kernel_size = 3, input_shape=[32, 32, 3]))

    # Flatten and fully connected layers
    model.add(tf.keras.layers.Flatten())

    # Output layer
    model.add(tf.keras.layers.Dense(10, activation='softmax'))

    model.compile(
        optimizer=optimizer,
        loss='sparse_categorical_crossentropy',
        metrics=['accuracy']
    )

    return model

## Train Model

### Find optimal number of epochs to train

In [16]:
model = build_model()

history = model.fit(train_images, train_labels, epochs=20, validation_split=0.2)

val_acc_per_epoch = history.history['val_accuracy']
best_epoch = val_acc_per_epoch.index(max(val_acc_per_epoch)) + 1
print('Best epoch: %d' % (best_epoch,))

/usr/local/lib/python3.11/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/20
1250/1250 ━━━━━━━━━━━━━━━━━━━━ 58s 44ms/step - accuracy: 0.3952 - loss: 1.9225 - val_accuracy: 0.6106 - val_loss: 1.0930
Epoch 2/20
1250/1250 ━━━━━━━━━━━━━━━━━━━━ 74s 39ms/step - accuracy: 0.6378 - loss: 1.0221 - val_accuracy: 0.6870 - val_loss: 0.8884
Epoch 3/20
1250/1250 ━━━━━━━━━━━━━━━━━━━━ 47s 38ms/step - accuracy: 0.7301 - loss: 0.7687 - val_accuracy: 0.7355 - val_loss: 0.7763
Epoch 4/20
1250/1250 ━━━━━━━━━━━━━━━━━━━━ 48s 39ms/step - accuracy: 0.7955 - loss: 0.5805 - val_accuracy: 0.7521 - val_loss: 0.7488
Epoch 5/20
1250/1250 ━━━━━━━━━━━━━━━━━━━━ 82s 39ms/step - accuracy: 0.8462 - loss: 0.4437 - val_accuracy: 0.7659 - val_loss: 0.7181
Epoch 6/20
1250/1250 ━━━━━━━━━━━━━━━━━━━━ 81s 39ms/step - accuracy: 0.8903 - loss: 0.3182 - val_accuracy: 0.7800 - val_loss: 0.6968
Epoch 7/20
1250/1250 ━━━━━━━━━━━━━━━━━━━━ 82s 39ms/step - accuracy: 0.9291 - loss: 0.2178 - val_accuracy: 0.7795 - val_loss: 0.7724
Epoch 8/20
1250/1250 ━━━━━━━━━━━━━━━━━━━━ 83s 39ms/step - accuracy: 0.9532 -

KeyboardInterrupt: 

### Train new model on optimal number of epochs

In [ ]:
optimal_epochs_model = build_model()

history = optimal_epochs_model.fit(train_images, train_labels, epochs=best_epoch, validation_split=0.2)

## Evaluate Model

In [17]:
model.evaluate(test_images, test_labels)

313/313 ━━━━━━━━━━━━━━━━━━━━ 5s 15ms/step - accuracy: 0.8005 - loss: 0.9366


[0.9461789131164551, 0.7975000143051147]

## Save Model

In [18]:
model.save('cifar_10_cnn.keras')